In [1]:
import time
import os
import pandas as pd
from datetime import datetime, timedelta
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [16]:
provinces = ['BKK','Chiangmai','Khonkaen','Rayong','Saraburi','Surat']
train_file_names = ['_train','_temp_surface','_weather_wind']
train_data_paths = []
train_data_dict = {}
lat_lon_dict = {'BKK':[13.729984,100.536443],'Chiangmai':[18.840633,98.969661],'Khonkaen':[16.445329,102.835251],'Rayong':[12.671521,101.275875],'Saraburi':[14.685833,100.871996],'Surat':[9.126057,99.325355]}

In [17]:
for province in provinces:
    train_data_paths.append('./'+province+'/train/')
    train_data_dict[province] = []
    for file_name in train_file_names:
        train_data_dict[province].append(train_data_paths[-1]+province.lower()+file_name+'.csv')

print(train_data_dict)

{'BKK': ['./BKK/train/bkk_train.csv', './BKK/train/bkk_temp_surface.csv', './BKK/train/bkk_weather_wind.csv'], 'Chiangmai': ['./Chiangmai/train/chiangmai_train.csv', './Chiangmai/train/chiangmai_temp_surface.csv', './Chiangmai/train/chiangmai_weather_wind.csv'], 'Khonkaen': ['./Khonkaen/train/khonkaen_train.csv', './Khonkaen/train/khonkaen_temp_surface.csv', './Khonkaen/train/khonkaen_weather_wind.csv'], 'Rayong': ['./Rayong/train/rayong_train.csv', './Rayong/train/rayong_temp_surface.csv', './Rayong/train/rayong_weather_wind.csv'], 'Saraburi': ['./Saraburi/train/saraburi_train.csv', './Saraburi/train/saraburi_temp_surface.csv', './Saraburi/train/saraburi_weather_wind.csv'], 'Surat': ['./Surat/train/surat_train.csv', './Surat/train/surat_temp_surface.csv', './Surat/train/surat_weather_wind.csv']}


In [40]:
for province in provinces:
    province_train = pd.read_csv(train_data_dict[province][0])
    province_temp_serface = pd.read_csv(train_data_dict[province][1])
    province_wheather_wind = pd.read_csv(train_data_dict[province][2])
    province_train = province_train.rename(columns={"Unnamed: 0": "date_time"})
    province_train['date_time'] = pd.to_datetime(province_train['date_time'])
    province_temp_serface['date_time'] = pd.to_datetime(province_temp_serface['date_time'])
    province_wheather_wind['date_time'] = pd.to_datetime(province_wheather_wind['date_time'])
    province_merge_temp_wind = pd.merge(province_temp_serface, province_wheather_wind, on='date_time')
    province_merge = pd.merge(province_train, province_merge_temp_wind, on='date_time',how='left')
    province_merge = province_merge.fillna(method='ffill')
    province_merge = province_merge.fillna(method='bfill')
    province_merge['lat'] = lat_lon_dict[province][0]
    province_merge['long'] = lat_lon_dict[province][1]
    province_merge.to_csv('./'+province+'/train/'+province.lower()+'_train_format.csv',index=False)

In [41]:
df = pd.read_csv('./'+'BKK'+'/train/'+'bkk'+'_train_format.csv')
df.head()

,date_time,PM2.5,temp,wind speed,wind dir,lat,long
0,2017-07-01 00:00:00,14.0,27.9,16.0,260.0,13.729984,100.536443
1,2017-07-01 01:00:00,10.0,27.9,16.0,260.0,13.729984,100.536443
2,2017-07-01 02:00:00,17.0,27.9,16.0,260.0,13.729984,100.536443
3,2017-07-01 03:00:00,20.0,27.9,16.0,260.0,13.729984,100.536443
4,2017-07-01 04:00:00,15.0,27.4,16.0,255.0,13.729984,100.536443


In [49]:
df_dict = {}
for province in provinces:
    df_dict[province]= pd.read_csv('./'+province+'/train/'+province.lower()+'_train_format.csv')

all_province = pd.concat(df_dict.values())
all_province = all_province.sort_values('date_time')
all_province.to_csv('all_province_train.csv',index=False)

In [45]:
for i in df_dict:
    print(i,df_dict[i].shape)

BKK (26304, 7)
Chiangmai (26304, 7)
Khonkaen (26304, 7)
Rayong (26304, 7)
Saraburi (26304, 7)
Surat (15336, 7)


In [55]:
data = pd.read_csv('all_province_train.csv')
data['date_time'] = pd.to_datetime(data['date_time'])
data['date_time']

0        2017-07-01 00:00:00
1        2017-07-01 00:00:00
2        2017-07-01 00:00:00
3        2017-07-01 00:00:00
4        2017-07-01 00:00:00
                 ...        
146851   2020-06-30 23:00:00
146852   2020-06-30 23:00:00
146853   2020-06-30 23:00:00
146854   2020-06-30 23:00:00
146855   2020-06-30 23:00:00
Name: date_time, Length: 146856, dtype: datetime64[ns]

In [53]:
data['date_time'].dt.hour

0          0
1          0
2          0
3          0
4          0
          ..
146851    23
146852    23
146853    23
146854    23
146855    23
Name: date_time, Length: 146856, dtype: int64

In [57]:
data['year'] = data['date_time'].dt.year
data['month'] = data['date_time'].dt.month
data['day'] = data['date_time'].dt.day
data['hour'] = data['date_time'].dt.hour
data = data[['year','month','day','hour','temp','wind speed','wind dir','lat','long','PM2.5']]
data

,year,month,day,hour,temp,wind speed,wind dir,lat,long,PM2.5
0,2017,7,1,0,27.9,16.0,260.0,13.729984,100.536443,14.0
1,2017,7,1,0,26.5,28.0,235.0,16.445329,102.835251,5.0
2,2017,7,1,0,27.9,21.0,250.0,12.671521,101.275875,10.0
3,2017,7,1,0,25.0,38.0,250.0,14.685833,100.871996,22.0
4,2017,7,1,0,23.2,32.0,235.0,18.840633,98.969661,13.0
...,...,...,...,...,...,...,...,...,...,...
146851,2020,6,30,23,25.9,41.0,250.0,14.685833,100.871996,7.0
146852,2020,6,30,23,28.5,47.0,270.0,12.671521,101.275875,7.0
146853,2020,6,30,23,25.2,17.0,225.0,18.840633,98.969661,18.0
146854,2020,6,30,23,29.6,21.0,230.0,16.445329,102.835251,13.0


In [58]:
data.to_csv('all_province_train_2.csv',index=False)